In [1]:
import os
from datetime import datetime

PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
MODEL_CHECKPOINTS_DIR = os.path.join(PROJECT_ROOT_DIR, "model_checkpoints")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)


import data_preparation as dp

#dp.generate_troubled_life_policy_data(no_of_policies=10000, runtime=5, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

#dp.generate_troubled_life_policy_data(no_of_policies=2000, runtime=5, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_train = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_test = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_length_train, max_policy_history_length_train = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_train)

policy_histories_length_test, max_policy_history_length_test = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_test)

max_policy_history_length = max(max_policy_history_length_train, max_policy_history_length_test)



# Pad the histories up to maximum length of both, train and test set

# policy_histories_train = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_train,
#                                           policy_histories_lengths=policy_histories_length_train,
#                                           max_policy_history_length=max_policy_history_length)
# 
# policy_histories_test = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_test,
#                                           policy_histories_lengths=policy_histories_length_test,
#                                           max_policy_history_length=max_policy_history_length)
# 
# # # Save padded data, since always generating and padding takes too long
# policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
# policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

# Extract features and labels from dataset as numpy.ndarray(s)
binary_classification = True

train_labels, train_features, train_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_policy_history_length,
                                       binary_classification=binary_classification)
test_labels, test_features, test_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_policy_history_length,
                                       binary_classification=binary_classification)

train_data = dp.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

In [2]:
import tensorflow as tf
import gan as gan


g_learning_rate = 0.01
d_real_learning_rate = 0.0001
d_fake_learning_rate = 0.0001
noise_dimensions = 2
batch_size = 200
g_n_outputs = 2
d_n_inputs = 2
d_n_outputs = 2 if binary_classification else max_policy_history_length

tf.reset_default_graph()
tf.set_random_seed(42)

noise = tf.placeholder(tf.float32, [batch_size, max_policy_history_length, noise_dimensions], name='noise')
noise_seq_length = tf.placeholder(tf.int32, [None], name='noise_seq_length')

X = tf.placeholder(tf.float32, [None, max_policy_history_length, d_n_inputs], name="X")
y = tf.placeholder(tf.int32, [None], name="y")
X_seq_length = tf.placeholder(tf.int32, [None], name="X_seq_length")

g_data = gan.generator(noise=noise, batch_size=batch_size, n_outputs=g_n_outputs, 
                       max_seq_length=max_policy_history_length, seq_length=noise_seq_length)

# # Let's make sure the initial value at [:, 0, 1] is 0 (current capital at policy inception)
# flat = tf.reshape(tensor=g_data, shape=[batch_size * max_policy_history_length * g_n_outputs])
# flat2 = tf.Variable(initial_value=tf.zeros_like(flat), trainable=False, name="xyFlat")
# flat3 = tf.assign(ref=flat2, value=flat)
# flat4 = tf.scatter_update(flat3, [1 + i * max_policy_history_length * g_n_outputs for i in range(batch_size)],
#                                  [0 for j in range(batch_size)])
# g_ms_data = tf.reshape(tensor=flat4, shape=[batch_size, max_policy_history_length, g_n_outputs])

logits_d_for_X, y_pred_X = gan.discriminator(X=X, seq_length=X_seq_length, n_outputs=d_n_outputs)
logits_d_for_g, y_pred_g = gan.discriminator(X=g_data, seq_length=noise_seq_length, n_outputs=d_n_outputs, reuse=True)

g_loss, accuracy_g = gan.generator_loss(batch_size=batch_size, logits_d=logits_d_for_g)
d_loss_real, accuracy_d_real = gan.discriminator_loss_real(logits=logits_d_for_X, y=y)
d_loss_fake, accuracy_d_fake = gan.discriminator_loss_fake(batch_size=batch_size, logits=logits_d_for_g)

g_trainer = gan.generator_trainer(g_learning_rate, g_loss)
d_trainer_real = gan.discriminator_trainer_real(d_real_learning_rate, d_loss_real + d_loss_fake)
#d_trainer_fake = gan.discriminator_trainer_fake(d_fake_learning_rate, d_loss_fake)

tf.summary.scalar('Generator_loss', g_loss)
tf.summary.scalar('Discriminator_loss_real', d_loss_real)
tf.summary.scalar('Discriminator_loss_fake', d_loss_fake)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

tvars = tf.trainable_variables()

d_saver = tf.train.Saver(var_list=[var for var in tvars if "t_discriminator" in var.name])
g_saver = tf.train.Saver(var_list=[var for var in tvars if "t_generator" in var.name])

In [3]:
import numpy as np

np.set_printoptions(formatter={'float_kind': (lambda x: "%.2f" % x)})

sess = tf.Session()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

# n_epochs = 100
# 
# # Pre-train discriminator
# for epoch in range(n_epochs):
#     for batch in range(train_data.num_examples // batch_size):
#         noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
#         noise_seq_length_batch = np.full(batch_size, max_policy_history_length)
# 
#         y_batch, X_batch, X_seq_length_batch = train_data.next_batch(batch_size)
# 
#         _, dLossReal, dLossFake, acc_real_train, acc_fake_train = \
#             sess.run([d_trainer_real, d_loss_real, d_loss_fake, accuracy_d_real, accuracy_d_fake],
#                      {X: X_batch, y: y_batch, X_seq_length: X_seq_length_batch,
#                       noise: noise_batch, noise_seq_length: noise_seq_length_batch})
# 
#     print("Epoch:", epoch, "dLossReal:", dLossReal, "acc_real_train:", acc_real_train,
#           "dLossFake:", dLossFake, "acc_fake_train:", acc_fake_train)
# 
# d_saver.save(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "discriminator.ckpt"))

In [5]:
d_saver.restore(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "discriminator.ckpt"))
#g_saver.restore(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "generator.ckpt"))

#Train generator and discriminator together
n_epochs = 200

for epoch in range(n_epochs):
    for batch in range(train_data.num_examples // batch_size):
        noise_batch = np.random.normal(1250, 5, size=[batch_size, max_policy_history_length, noise_dimensions])
        noise_batch[:, 0, 1] = 0
        noise_seq_length_batch = np.full(batch_size, max_policy_history_length)
        
        y_batch, X_batch, X_seq_length_batch = train_data.next_batch(batch_size)

        # Train discriminator on both real and fake data
        _, dLossReal, dLossFake, acc_real_train, acc_fake_train = \
            sess.run([d_trainer_real, d_loss_real, d_loss_fake, accuracy_d_real, accuracy_d_fake],
                     {X: X_batch, y: y_batch, X_seq_length: X_seq_length_batch,
                      noise: noise_batch, noise_seq_length: noise_seq_length_batch})

        # Train generator    
        _, dLossFake, acc_d_fake, gLoss, acc_g, gData = sess.run([g_trainer, d_loss_fake, accuracy_d_fake,
                                                                         g_loss, accuracy_g, g_data],
                                          feed_dict={noise: noise_batch, noise_seq_length: noise_seq_length_batch})

    print("Epoch:", epoch, 
          "dLossFake:", dLossFake, "acc_fake_train:", acc_d_fake,
          "gLoss:", gLoss, "acc_g:", acc_g)

    print(gData[0])


INFO:tensorflow:Restoring parameters from /Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/model_checkpoints/discriminator.ckpt


Epoch: 0 dLossFake: 0.17438 acc_fake_train: 1.0 gLoss: 1.83245 acc_g: 0.0
[[65.65 118.47]
 [111.03 302.92]
 [41.70 233.71]
 [67.19 135.30]
 [59.82 97.27]
 [27.55 65.47]
 [20.22 52.06]
 [19.23 51.36]
 [18.18 51.21]
 [16.75 49.42]
 [15.55 47.00]
 [14.50 45.50]
 [13.73 44.19]
 [13.12 43.12]
 [12.66 41.79]]


Epoch: 1 dLossFake: 0.268625 acc_fake_train: 1.0 gLoss: 1.44576 acc_g: 0.0
[[4.44 2.27]
 [4.84 4.45]
 [4.09 8.43]
 [5.05 9.05]
 [5.32 9.85]
 [4.49 10.91]
 [4.60 11.66]
 [3.85 10.75]
 [4.03 11.80]
 [3.97 11.76]
 [3.96 11.77]
 [3.95 11.66]
 [3.88 11.57]
 [3.92 11.70]
 [3.91 11.71]]


Epoch: 2 dLossFake: 0.0518422 acc_fake_train: 1.0 gLoss: 2.98573 acc_g: 0.0
[[4.48 4.01]
 [4.14 5.54]
 [4.43 6.33]
 [7.41 11.57]
 [5.67 12.34]
 [6.84 14.56]
 [9.52 15.13]
 [10.38 18.76]
 [8.00 21.14]
 [7.13 26.44]
 [7.72 32.88]
 [8.87 39.03]
 [9.42 38.11]
 [8.44 33.10]
 [7.06 29.39]]


Epoch: 3 dLossFake: 0.602505 acc_fake_train: 1.0 gLoss: 0.792848 acc_g: 0.0
[[13.74 20.41]
 [18.93 27.08]
 [16.41 24.08]
 [17.25 14.64]
 [10.59 33.85]
 [19.96 39.88]
 [22.20 47.11]
 [28.74 71.21]
 [9.30 88.34]
 [25.48 73.33]
 [20.96 89.79]
 [33.41 107.17]
 [-4.10 11.41]
 [-4.97 -6.71]
 [-1.15 -3.97]]


...

Epoch: 29 dLossFake: 0.0151279 acc_fake_train: 1.0 gLoss: 4.19877 acc_g: 0.0
[[14.21 7.08]
 [18.87 10.77]
 [17.91 25.34]
 [18.76 21.40]
 [16.37 34.13]
 [26.69 37.66]
 [23.87 48.15]
 [23.46 60.75]
 [27.33 62.57]
 [30.09 69.74]
 [24.28 86.23]
 [27.42 97.44]
 [39.63 119.94]
 [43.83 136.65]
 [43.02 135.15]]


Epoch: 30 dLossFake: 0.0956039 acc_fake_train: 1.0 gLoss: 2.39497 acc_g: 0.0
[[11.12 5.04]
 [20.01 7.09]
 [16.16 21.07]
 [18.78 21.83]
 [19.28 33.42]
 [26.97 39.63]
 [29.95 50.57]
 [27.99 72.27]
 [31.07 74.64]
 [34.45 85.79]
 [30.77 107.18]
 [36.60 120.40]
 [51.06 151.71]
 [57.22 174.00]
 [58.12 180.33]]


Epoch: 31 dLossFake: 0.0881288 acc_fake_train: 1.0 gLoss: 2.47271 acc_g: 0.0
[[16.63 6.91]
 [31.27 7.08]
 [23.83 29.02]
 [30.73 28.48]
 [26.56 50.95]
 [34.62 66.37]
 [42.25 70.75]
 [43.84 104.67]
 [39.50 106.49]
 [48.76 117.36]
 [46.59 150.42]
 [49.53 162.73]
 [65.67 200.33]
 [71.20 225.06]
 [72.51 236.29]]


Epoch: 32 dLossFake: 0.0121928 acc_fake_train: 1.0 gLoss: 4.41301 acc_g: 0.0
[[18.32 9.05]
 [23.06 15.82]
 [19.22 30.17]
 [28.23 33.57]
 [24.73 55.33]
 [29.27 59.46]
 [31.65 68.44]
 [30.20 77.19]
 [36.65 86.91]
 [32.88 84.32]
 [29.40 110.37]
 [36.14 115.07]
 [41.67 117.20]
 [33.08 148.37]
 [36.72 149.19]]


KeyboardInterrupt: 

In [5]:
g_saver.save(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "generator.ckpt"))

'/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife/model_checkpoints/generator.ckpt'